In [1]:
optimizer: str = 'adam'
epochs: int = 5
batch_size: int = 64

In [2]:
import tensorflow as tf
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import layers, models

# Load the California housing dataset
california_housing = fetch_california_housing()
X = california_housing.data
y = california_housing.target

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features by removing the mean and scaling to unit variance
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

/Users/jules/Projects/notebooks/env/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
# Build the regression model
model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(1))  # Linear activation for regression

In [4]:
# Compile the model
model.compile(optimizer=optimizer, loss='mean_squared_error')

In [5]:
from codecarbon import EmissionsTracker
from examples.serializer import Serializer

# Serialize the model
serializer = Serializer()
row = serializer.serialize(model, epochs, batch_size)

# Start the emissions tracker
tracker = EmissionsTracker(log_level='critical')
tracker.start()

# Train the model
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.1)

# Save the model carbon emissions
row['co2'] = tracker.stop()

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'get_non_trainable_parameters', 'get_trainable_parameters', 'serialize']
Epoch 1/5
233/233 [==============================] - 0s 845us/step - loss: 1.1221 - val_loss: 0.5289
Epoch 2/5
233/233 [==============================] - 0s 723us/step - loss: 0.4542 - val_loss: 0.4406
Epoch 3/5
233/233 [==============================] - 0s 611us/step - loss: 0.3921 - val_loss: 0.4215
Epoch 4/5
233/233 [==============================] - 0s 595us/step - loss: 0.3700 - val_loss: 0.4094
Epoch 5/5
233/233 [==============================] - 0s 585us/step - loss: 0.3596 - val_loss: 0.4061


In [6]:
# Evaluate the model on the test set
test_loss = model.evaluate(X_test, y_test)
print(f'Test mean squared error: {test_loss}')

129/129 [==============================] - 0s 358us/step - loss: 0.3682
Test mean squared error: 0.3681676685810089


In [7]:
from csv import DictWriter

# Save the model metadata
with open('results.csv', 'a') as f:
    writer = DictWriter(f, fieldnames=row.keys())
    writer.writerow(row)